In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error


# TP1 Aprendizaje Profundo

Se desea construir un sistema de recomendación de películas. Para esto se cuenta con un dataset de las puntuaciones que los usuarios han asignado a las peliculas disponibles.

Link dataset: https://drive.google.com/file/d/1Og9H-8oqb3_Wo_WOakeAuRR_mwr922Ar/view?usp=sharing

Para verificar la factibilidad del proyecto con datos válidos, se decide utilizar solamente las 200 películas con más votos y sobre eso los usuarios que han puntuado al menos 100 películas.

1- Analizar el dataset para utilizar solamente las 200 películas con mayor cantidad de votos y los usuarios que hayan votado al menos 100 películas.

2- A partir del dataset del punto 1, construir una única red neuronal que utilice una capa de embeddings para el id de usuario, una capa de embeddings para el id de película y al menos dos capas lineales que sea capaz de predecir el puntaje que cada usuario colocó a cada pelicula. Usar tecnicas de normalizacion en caso de ser necesario.

3- Graficar las evoluciones de las funciones de costo en entrenamiento y validacion, como asi tambien las metricas de validacion. Explicar los resultados obtenidos.

4- Construir una funcion capaz de recibir un usuario al azar, una cantidad "p" de películas que dicho usuario haya puntuado y verificar la predicción del modelo. Comparar con los puntajes reales contra los que el usuario asignó a dicha/s película/s.

5- Contruir una funcion capaz de realizar una recomendación de película para un usuario determinado utilizando los embeddings de usuario o los embeddings de películas. Comprobar si la recomendación es correcta haciendo una predicción del puntuaje con la red neuronal.

6- Con el mejor modelo obtenido del punto 2, elegir al menos 3 hiperparametros y aplicar algun metodo de tuneo. Explicar resultados obtenidos.

## 1. Análisis exploratorio

Primero analicemos el dataset y preparemos los datos para el entrenamiento con aprendizaje profundo.

In [34]:
#Cargar el dataset
movies_df = pd.read_csv("./datasets/ratings.csv")

In [35]:
movies_df.describe()

,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


In [36]:
movies_df.head(10)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
5,1,70,3.0,964982400
6,1,101,5.0,964980868
7,1,110,4.0,964982176
8,1,151,5.0,964984041
9,1,157,5.0,964984100


In [37]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


Estamos en frente a un dataset con tan solo 4 features y una gran cantidad de entradas (100836). En particular se observan dos features "userID" y "movieID" que hacen referencia a usuarios X y películas Y respectivamente. Es un caso donde estos features si bien son numéricos se refieren más correctamente a variables categoricas, por lo que el uso de la herramienta de embeddings resulta de mucha utilidad.

Además de estos features "rating" es un flotante que va de 0 a 5 (y el feature objetivo en este caso) y "timestamp" parece ser una medida de cuando fue subida la review que a nivel lógico no parecería aportar mucha información

De acuerdo a lo que pide el problema debemos filtrar el dataset para quedarnos solo con las 200 películas más votadas y sobre esas con los usuarios que hayan votado más de 100 películas.

In [38]:
#Filtrar las 200 películas más rateadas

#Cuento las instancias de cada película
value_counts_movies = movies_df["movieId"].value_counts()

#Guardo los id de las 200 películas con más ratings
top_200_movies = value_counts_movies.head(200).index

#Filtro el dataset para que solo contenga estas películas
movies_df = movies_df[movies_df["movieId"].isin(top_200_movies)]

movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25764 entries, 0 to 100452
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   userId     25764 non-null  int64  
 1   movieId    25764 non-null  int64  
 2   rating     25764 non-null  float64
 3   timestamp  25764 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 1006.4 KB


In [39]:
#Chequeo que mi dataset solo contenga 200 películas diferentes
movies_df["movieId"].value_counts()

movieId
356     329
318     317
296     307
593     279
2571    278
       ... 
3897     83
1101     83
16       82
788      82
1584     82
Name: count, Length: 200, dtype: int64

Misma idea pero para los usuarios ahora

In [40]:
#Filtrar los 100 usuarios que más reviews tienen

#Cuento la cantidad de reviews por user
value_counts_users = movies_df["userId"].value_counts()

#Guardo los id de las 100 users con más reviewss
top_users = value_counts_users[value_counts_users>99].index

#Filtro el dataset para que solo contenga estos usuarios
movies_df = movies_df[movies_df["userId"].isin(top_users)]



In [41]:
#Chequeo que no hayan quedado usuarios con menos de 100 reviews dentro del dataset
movies_df["userId"].value_counts()


userId
414    194
599    189
68     185
480    177
474    173
      ... 
200    104
453    103
166    102
603    102
354    100
Name: count, Length: 63, dtype: int64

In [42]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8329 entries, 1772 to 100452
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   userId     8329 non-null   int64  
 1   movieId    8329 non-null   int64  
 2   rating     8329 non-null   float64
 3   timestamp  8329 non-null   int64  
dtypes: float64(1), int64(3)
memory usage: 325.4 KB


## 2. Separación Train-Test-Validation

Separamos en train test el dataset, también me quito timestamp porque considero que no aporta información

In [43]:
# Guardo user_id para modelo con embeddings
user_id = movies_df['userId']

# Guardo movie_id para modelo con embeddings
movie_id = movies_df['movieId']

X = movies_df.drop(columns=["rating","timestamp"]).values
y = movies_df["rating"].values


In [44]:
#Separamos en los sets de entrenamiento y evaluación
X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y,np.arange(X.shape[0]), test_size = 0.20, random_state = 42)

#Separamos sobre el set de entrenamiento, un subset de validación
X_train, X_valid, y_train, y_valid, train_idx, valid_idx = train_test_split(X_train, y_train, train_idx, test_size = 0.15, random_state = 42)

In [45]:
n_train = X_train.shape[0]
n_test = X_test.shape[0]
n_valid = X_valid.shape[0]

#Chequeamos que el total de datos está comprendido en la separación
n_test+n_train+n_valid

8329

## 3. Pytorch

In [46]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn import metrics

In [47]:
# Transformo user id a indices (idx) consecutivos para utilizar embeddings
user_id_to_idx = {value:i for i,value in enumerate(user_id.unique())}

# Transformo user id a indices (idx) consecutivos para utilizar embeddings
movie_id_to_idx = {value:i for i,value in enumerate(movie_id.unique())}

In [48]:
# Vector de user_idx en el dataset
user_idx = np.array([user_id_to_idx[value] for value in user_id])

# Vector de movie_idx en el dataset
movie_idx = np.array([movie_id_to_idx[value] for value in movie_id])

In [49]:
# Divido el vector user_idx en entrenamiento y validación
user_idx_train = user_idx[train_idx]
user_idx_valid = user_idx[valid_idx]

# Divido el vector movie_idx en entrenamiento y validación
movie_idx_train = movie_idx[train_idx]
movie_idx_valid = movie_idx[valid_idx]


In [75]:
# Pytorch necesita de una clase de dataset que extienda de torch.utils.data.Dataset
# Esta clase dataset debe sobreescribir los métodos init, len y getitem
class MyDatasetWithEmbddings(Dataset):

  #__init__ guarda el dataset en una variable de clase
  def __init__(self, user_idx, movie_idx, y): #no tengo features más alla de los embeddings
    self.user_idx = user_idx
    self.movie_idx = movie_idx
    self.y = y

  # __len__ define el comportamiento de la función len() sobre el objeto
  def __len__(self):
    return self.user_idx.shape[0]

  # __getitem__ define el comportamiento de los []
  def __getitem__(self, idx):
    return  self.user_idx[idx], self.movie_idx[idx], self.y[idx]

In [76]:
# Creo el dataset de entrenamiento
df_train = MyDatasetWithEmbddings(user_idx_train, movie_idx_train, y_train)
# Creo el dataset de validación
df_valid = MyDatasetWithEmbddings(user_idx_valid, movie_idx_valid, y_valid)

In [77]:
# Pytorch utiliza DataLoader para entregar los dataset de a batches
train_dataloader = DataLoader(df_train, batch_size = 64, shuffle= True)
valid_dataloader = DataLoader(df_valid, batch_size=64)

Ahora tenemos que definir la arquitectura de nuestra red neuronal. De acuerdo a lo que pide el ejercicio tenemos que definir una capa de embedding por feature y luego dos capas lineales.

En particular y como no tengo referencia del problema voy a usar una regla empirica para definir la dimensionalidad de la capa de embeddings, en particular tomar aproximadamente la raíz cuadrada de la cantidad de instancias distintas. Esto es 14 para moviesId y 8 para userId.

En cuanto a las capas lineales mantendré las estándar mostradas en clase.

In [78]:
# Arquitectura con embeddings
class NNetWithEmbeddings(torch.nn.Module):

  def __init__(self):
    super().__init__()
    self.embeddings_user = torch.nn.Embedding(num_embeddings=63, embedding_dim=8)
    self.embeddings_movie = torch.nn.Embedding(num_embeddings=200, embedding_dim=14)
    self.linear_1 = torch.nn.Linear(in_features=14+8, out_features=200, bias=True)
    self.relu_1 = torch.nn.ReLU()
    self.linear_2 = torch.nn.Linear(in_features = 200, out_features=100, bias=True)
    self.relu_2 = torch.nn.ReLU()
    self.output = torch.nn.Linear(in_features = 100, out_features= 1, bias=True)

  def forward(self, user_idx, movie_idx):
    embeddings_outputs_user = self.embeddings_user(user_idx)
    embeddings_outputs_movie =self.embeddings_movie(movie_idx)
    x = torch.cat([embeddings_outputs_user, embeddings_outputs_movie], dim=1)
    x = self.linear_1(x)
    x = self.relu_1(x)
    x = self.linear_2(x)
    x = self.relu_2(x)
    x = self.output(x)
    return x

In [79]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [80]:
#Inicializo mi red
nnnetWithEmbeddings = NNetWithEmbeddings()
nnnetWithEmbeddings = nnnetWithEmbeddings.to(device)

In [89]:
loss_function = torch.nn.MSELoss()
# Optimizador con regularización L2 (parámetro weight_decay)
optimizer = torch.optim.Adam(nnnetWithEmbeddings.parameters(), lr=0.001, weight_decay=0.00001) 

In [90]:
# Mini-Batch

# cantidad de epochs
epochs = 100

train_loss_by_epoch=[]
valid_loss_by_epoch=[]

# Doble loop algoritmo Mini-Batch
for epoch in range(epochs):
  
  ############################################
  ## Entrenamiento
  ############################################
  nnnetWithEmbeddings.train(True)

  epoch_loss = 0
  epoch_y_hat = []
  epoch_y = []
  
  for i,data in enumerate(train_dataloader):
    # Obtengo los datos del batch de entrenamiento
    embed_user_batch, embed_movie_batch, y_batch = data
    # Copio el batch al dispositivo donde entreno la red neuronal
    embed_user_batch = embed_user_batch.to(device).int()
    embed_movie_batch = embed_movie_batch.to(device).int()
    y_batch = y_batch.to(device).float().reshape(-1, 1)

    # Paso forward
    # Limpio optimizer para empezar un nuevo cálculo de gradiente
    optimizer.zero_grad()
    nnet_output = nnnetWithEmbeddings(embed_user_batch, embed_movie_batch)
    y_batch_hat = nnet_output
    
    # Calculo el loss
    loss = loss_function(nnet_output, y_batch)

    # Backpropagation
    loss.backward()

    # Actualizar los parámetros
    optimizer.step()

    # Almaceno los valores reales y mis predicciones para cálcular las métricas
    epoch_y += list(y_batch.detach().cpu().numpy())
    epoch_y_hat += list(y_batch_hat.detach().cpu().numpy())
    # Acumulo la loss del batch
    epoch_loss = epoch_loss + loss.item()

    # Almaceno la loss de la epoch para graficar
  train_loss_by_epoch.append(epoch_loss)
  # Cálculo la métrica de la epoch
  train_mse = mean_squared_error(epoch_y, epoch_y_hat)

  ############################################
  ## Validación
  ############################################
  # Desactivo el cálculo de gradiente para validación
  nnnetWithEmbeddings.train(False)

  valid_epoch_loss = 0
  valid_epoch_y_hat = []
  valid_epoch_y = []

  for i,data in enumerate(valid_dataloader):
    # Obtengo los datos del batch de validación
    embed_user_batch, embed_movie_batch, y_batch = data
    # Copio el batch al dispositivo donde entreno la red neuronal
    embed_user_batch = embed_user_batch.to(device).int()
    embed_movie_batch = embed_movie_batch.to(device).int()
    y_batch = y_batch.to(device).float().reshape(-1, 1)

    # Paso forward
    nnet_output = nnnetWithEmbeddings(embed_user_batch, embed_movie_batch)
    y_batch_hat = nnet_output
    
    # Calculo el loss
    loss = loss_function(nnet_output, y_batch)

    # En validación no hago backpropagation!!

    # Almaceno los valores reales y mis predicciones para cálcular las métricas
    valid_epoch_y += list(y_batch.detach().cpu().numpy())
    valid_epoch_y_hat += list(y_batch_hat.detach().cpu().numpy())
    # Acumulo la loss del batch
    valid_epoch_loss = valid_epoch_loss + loss.item()

  # Calculo la media de la loss
  valid_epoch_loss = valid_epoch_loss / n_valid
  # Almaceno la loss de la epoch para graficar
  valid_loss_by_epoch.append(valid_epoch_loss)
  # Cálculo la métrica de la epoch
  valid_mse = mean_squared_error(valid_epoch_y, valid_epoch_y_hat)

  ############################################
  ## Impresión de resultados por epoch
  ############################################
  print(f" Epoch {epoch} | " \
        f"Train/Valid loss: {epoch_loss:.3f} / {valid_epoch_loss:.3f} | " \
        f"Train/Valid mse: {train_mse:.3f} / {valid_mse:.3f}")

 Epoch 0 | Train/Valid loss: 12107.455 / 279.382 | Train/Valid mse: 12107.454 / 279.382
 Epoch 1 | Train/Valid loss: 216.692 / 191.593 | Train/Valid mse: 216.692 / 191.593
 Epoch 2 | Train/Valid loss: 146.488 / 141.449 | Train/Valid mse: 146.488 / 141.449
 Epoch 3 | Train/Valid loss: 111.012 / 148.344 | Train/Valid mse: 111.012 / 148.344
 Epoch 4 | Train/Valid loss: 98.445 / 103.008 | Train/Valid mse: 98.445 / 103.008
 Epoch 5 | Train/Valid loss: 79.785 / 101.061 | Train/Valid mse: 79.785 / 101.061
 Epoch 6 | Train/Valid loss: 70.160 / 118.698 | Train/Valid mse: 70.160 / 118.698
 Epoch 7 | Train/Valid loss: 66.469 / 80.960 | Train/Valid mse: 66.469 / 80.960
 Epoch 8 | Train/Valid loss: 59.602 / 75.755 | Train/Valid mse: 59.602 / 75.755
 Epoch 9 | Train/Valid loss: 56.552 / 58.369 | Train/Valid mse: 56.552 / 58.369
 Epoch 10 | Train/Valid loss: 48.025 / 53.096 | Train/Valid mse: 48.025 / 53.096
 Epoch 11 | Train/Valid loss: 41.988 / 54.418 | Train/Valid mse: 41.988 / 54.418
 Epoch 12 | 